In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data=pd.read_csv("/kaggle/input/titanic/train.csv")
data['Age']=data["Age"].fillna(data['Age'].mean())
print(data.describe())
print(data.head)

In [ ]:
# lets get the visualisation of the data by plotting different graphs and figures
data['dead']=1-data['Survived']
data.groupby('Sex').mean()[['Survived','dead']].plot(kind='bar',stacked=True,colors=['g','r'])

In [ ]:
fig=plt.figure(figsize=(25,7))
sns.violinplot(x='Sex',y='Age',hue='Survived',data=data,split=True,palette={0:'r',1:'g'})


In [ ]:
figure=plt.figure(figsize=(25,7))
plt.hist([data[data['Survived']==1]['Fare'],data[data['Survived']==0]['Fare']]
         ,stacked=True,color=['g','r'],
         bins=50,label=['Survived','dead'])
plt.xlabel('fare')
plt.ylabel('No/. of passenger')
plt.legend()

## lets combine the age survived and fare


In [ ]:
plt.figure(figsize=(22,7))
ax=plt.subplot()
ax.scatter(data[data['Survived']==1]['Age'],data[data['Survived']==1]['Fare'],c='green',s=data[data['Survived']==1]['Fare'])
ax.scatter(data[data['Survived']==0]['Age'],data[data['Survived']==0]['Fare'],c='red',s=data[data['Survived']==0]['Fare'])

In [ ]:
#Fare corelates with the class below

In [ ]:
ax=plt.subplot()
ax.set_ylabel('Average fare')
data.groupby('Pclass').mean()['Fare'].plot(kind='bar',figsize=(25,7))

In [ ]:
#similarly see the embarkation sideeffect 

In [ ]:
fig=plt.figure()
sns.violinplot(x='Embarked',y='Fare',hue='Survived',data=data,split=True,palette={0:'r',1:'g'})

**Feature Engineering**********
extracting the passengers title

In [ ]:
def get_combined_data():
    train=pd.read_csv("/kaggle/input/titanic/train.csv")
    test=pd.read_csv("/kaggle/input/titanic/test.csv")
    targets=train.Survived
    train.drop(['Survived'],1,inplace=True)
    combined=train.append(test)
    combined.reset_index(inplace=True)
  
    return combined
combined=get_combined_data()



In [ ]:
titles=set()
for name in data['Name']:
    titles.add(name.split(',')[1].split('.')[0].strip())
title_dict={
    "Lady":"Royality",
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master"
           }
def get_titles():
    combined['Title']=combined['Name'].map(lambda name: name.split(',')[1].split('.')[0].strip())
    combined['Title']=combined.Title.map(title_dict)

    return combined
combined=get_titles()
print(combined.head)
    

In [ ]:
# Processing Ages
#print(combined.iloc[:891].Age.isnull().sum())
#print(combined.iloc[891:].Age.isnull().sum())
grouped_train=combined.iloc[:891].groupby(['Sex','Pclass','Title'])
grouped_train_median=grouped_train.median()
grouped_train_median=grouped_train_median.reset_index()[['Sex','Pclass','Title','Age']]
print(grouped_train_median.head())
def fillage(row):
    condition=(grouped_train_median['Sex']==row['Sex'])&(grouped_train_median['Title']==row['Title'])&(grouped_train_median['Pclass']==row['Pclass'])
    return grouped_train_median[condition]['Age'].values[0]
def process_age():
    global combined
    combined['Age']=combined.apply(lambda row:fillage(row) if np.isnan(row['Age']) else row['Age'],axis=1)
    return combined
combined=process_age()
print(combined['Age'].isnull)


 **Prcoess name**

In [ ]:
def evaluate_name():
    global combined
    combined.drop('Name',axis=1,inplace=True)
    dummies=pd.get_dummies(combined['Title'],prefix='Title')
    combined=pd.concat([combined,dummies],axis=1)
    combined.drop('Title',inplace=True,axis=1)
    return combined
combined=evaluate_name()
print(combined.head())


**process Sex**

In [ ]:
def processsex():
    global combined
    combined['Sex']=combined['Sex'].map({'male':1,'female':0})
    return combined
combined=processsex()
print(combined.head())

**Process Fare**

In [ ]:
def Processfare():
    global combined
    combined.Fare.fillna(combined.iloc[:891].Fare.mean(),inplace=True)
    return combined
combined=Processfare()
print(combined['Fare'].isnull)

**Processing Embarked**

In [ ]:
def Process_embarked():
    global combined
    combined.Embarked.fillna('S',inplace=True)
    dummies=pd.get_dummies(combined['Embarked'],prefix='embark')
    combined=pd.concat([combined,dummies],axis=1)
    combined.drop('Embarked',axis=1,inplace=True)
    return combined
combined=Process_embarked()
print(combined.head())

**processing cabin******

In [ ]:
train_cabin,test_cabin=set(),set()
for c in combined.iloc[:891]['Cabin']:
    try:
        train_cabin.add(c[0])
    except:
        train_cabin.add('U')
for c in combined.iloc[891:]['Cabin']:
    try:
        test_cabin.add(c[0])
    except:
        test_cabin.add('U')        
print(train_cabin)
print(test_cabin)
def processcabin():
    global combined
    combined.Cabin.fillna('U',inplace=True)
    combined['Cabin']=combined['Cabin'].map(lambda c:c[0])
    dummy=pd.get_dummies(combined['Cabin'],prefix='Cabin')
    combined=pd.concat([combined,dummy],axis=1)
    combined.drop('Cabin',axis=1,inplace=True)
    return combined
combined=processcabin()
print(combined.head())

**Processing Pclasss**

In [ ]:
def processingPclass():
    global combined
    dummy=pd.get_dummies(combined['Pclass'],prefix='Class')
    combined.drop('Pclass',axis=1,inplace=True)
    combined=pd.concat([combined,dummy],axis=1)
    return combined
combined=processingPclass()
print(combined.head())


**Processing Ticket**

In [ ]:
def filterticket(ticket):
    ticket=ticket.replace('/','')
    ticket=ticket.replace('.','')
    ticket=ticket.split()
    ticket=map(lambda t: t.strip(), ticket)
    ticket=list(filter(lambda t:  not t.isdigit(),ticket))
    if (len(ticket)>0):
        return ticket[0]
    else:
        return 'XXX'
tickets=set()    
for t in combined['Ticket']:
    tickets.add(filterticket(t))
print(tickets)    
def processtickets():
    global combined
    def filterticket(ticket):
        ticket=ticket.replace('/','')
        ticket=ticket.replace('.','')
        ticket=ticket.split()
        ticket=map(lambda t: t.strip(), ticket)
        ticket=list(filter(lambda t:  not t.isdigit(),ticket))
        if (len(ticket)>0):
            return ticket[0]
        else:
            return 'XXX' 
    combined['Ticket']=combined['Ticket'].map(filterticket)
    dummy=pd.get_dummies(combined['Ticket'],prefix='Ticket')

    combined.drop('Ticket',axis=1,inplace=True)
    combined=pd.concat([combined,dummy],axis=1)
    return combined
combined=processtickets()
print(combined.head())
    
    
    

Process Family

In [ ]:
def Processfamily():
    global combined
    combined['Familysize']=combined['SibSp']+combined['Parch']+1
    combined['Single']=combined['Familysize'].map(lambda s: 1 if s==1 else 0)
    combined['Small']=combined['Familysize'].map(lambda s: 1 if s>=2 and s<=4 else 0)
    combined['big']=combined['Familysize'].map(lambda s: 1 if s>4  else 0)
    return combined
combined=Processfamily()
combined.drop('SibSp',axis=1,inplace=True)
combined.drop('Parch',axis=1,inplace=True)
print(combined.head())
    

> Modeling

In [ ]:
# 1. breaking the train and test from combined data
def recovery():
    global combined
    target=pd.read_csv("/kaggle/input/titanic/train.csv",usecols=['Survived'])['Survived'].values
    train=combined.iloc[:891]
    test=combined.iloc[891:]
    return train,test,target
train,test,target=recovery()
    

In [ ]:
#2 we import all the necessary libraries
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV

In [ ]:
clf=RandomForestClassifier(n_estimators=50,max_features='sqrt')
clf=clf.fit(train,target)

In [ ]:
features=pd.DataFrame()
features['feature']=train.columns
features['importance']=clf.feature_importances_
features.sort_values(by=['importance'],ascending=True,inplace=True)
features.set_index('feature',inplace=True)
features.plot(kind='barh',figsize=(25,35))

In [ ]:
model=SelectFromModel(clf,prefit=True)
train_reduced=model.transform(train)
print(train_reduced.shape)
test_reduced=model.transform(test)
print(test_reduced.shape)

In [ ]:
#3 lets try different models
def compute_score(clf,x,y,scoring='accuracy'):
    xval=cross_val_score(clf,x,y,cv=5,scoring=scoring)
    return np.mean(xval)

logreg=LogisticRegression()
logregcv=LogisticRegressionCV()
rf=RandomForestClassifier()
gboost=GradientBoostingClassifier()
models=[logreg,logregcv,rf,gboost]
for model in models:
    print("cross validation of:{0}".format(model.__class__))
    score=compute_score(clf=model,x=train_reduced,y=target,scoring='accuracy')
    print("cv score{0}".format(score))


In [ ]:
#turn run_gs to True if you want to run the gridsearch again.
run_gs = False

if run_gs:
    parameter_grid = {
                 'max_depth' : [4, 6, 8],
                 'n_estimators': [50, 10],
                 'max_features': ['sqrt', 'auto', 'log2'],
                 'min_samples_split': [2, 3, 10],
                 'min_samples_leaf': [1, 3, 10],
                 'bootstrap': [True, False],
                 }
    forest = RandomForestClassifier()
    cross_validation = StratifiedKFold(n_splits=5)

    grid_search = GridSearchCV(forest,
                               scoring='accuracy',
                               param_grid=parameter_grid,
                               cv=cross_validation,
                               verbose=1
                              )

    grid_search.fit(train, target)
    model = grid_search
    parameters = grid_search.best_params_

    print('Best score: {}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))
    
else: 
    parameters = {'bootstrap': False, 'min_samples_leaf': 3, 'n_estimators': 50, 
                  'min_samples_split': 10, 'max_features': 'sqrt', 'max_depth': 6}
    
    model = RandomForestClassifier(**parameters)
    model.fit(train, target)

In [ ]:
output = model.predict(test).astype(int)
df_output = pd.DataFrame()
aux = pd.read_csv('/kaggle/input/titanic/test.csv')
df_output['PassengerId'] = aux['PassengerId']
df_output['Survived'] = output
df_output[['PassengerId','Survived']].to_csv('submission_test.csv', index=False)


In [ ]:
check_submission = pd.read_csv('submission_test.csv')
print(check_submission)